<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# TODO - Words, words, mere words, no matter from the heart.
import pandas as pd
import numpy as np
import random 
import sys
import os
import requests
import re

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense, Embedding, Dropout, SimpleRNN, LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import LambdaCallback


In [0]:
#load data
data = requests.get('https://www.gutenberg.org/files/100/100-0.txt')
poem = data.text

In [5]:
head, text = poem.split('1\r\n\r\n', 1)
head

'ï»¿\r\nProject Gutenbergâ\x80\x99s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, youâ\x80\x99ll\r\nhave to check the laws of the country where you are located before using\r\nthis ebook.\r\n\r\n\r\nTitle: The Complete Works of William Shakespeare\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: January 1994 [EBook #100]\r\nLast Updated: November 7, 2019\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***\r\n\r\n\r\n\r\n\r\nThe Complete Works of William Shakespeare\r\n\r\n\r\n\r\nby William

In [6]:
text[:1000]

'From fairest creatures we desire increase,\r\nThat thereby beautyâ\x80\x99s rose might never die,\r\nBut as the riper should by time decease,\r\nHis tender heir might bear his memory:\r\nBut thou contracted to thine own bright eyes,\r\nFeedâ\x80\x99st thy lightâ\x80\x99s flame with self-substantial fuel,\r\nMaking a famine where abundance lies,\r\nThy self thy foe, to thy sweet self too cruel:\r\nThou that art now the worldâ\x80\x99s fresh ornament,\r\nAnd only herald to the gaudy spring,\r\nWithin thine own bud buriest thy content,\r\nAnd, tender churl, makâ\x80\x99st waste in niggarding:\r\n  Pity the world, or else this glutton be,\r\n  To eat the worldâ\x80\x99s due, by the grave and thee.\r\n\r\n\r\n                    2\r\n\r\nWhen forty winters shall besiege thy brow,\r\nAnd dig deep trenches in thy beautyâ\x80\x99s field,\r\nThy youthâ\x80\x99s proud livery so gazed on now,\r\nWill be a tattered weed of small worth held:\r\nThen being asked, where all thy beauty lies,\r\nWhere

In [7]:
len(text)

5774402

In [8]:
text = text.replace('\r\n', ' ')
text = text.replace('â\x80\x99', "'")
text = text.replace('â\x80\x98', "'")
text[:2500]

"From fairest creatures we desire increase, That thereby beauty's rose might never die, But as the riper should by time decease, His tender heir might bear his memory: But thou contracted to thine own bright eyes, Feed'st thy light's flame with self-substantial fuel, Making a famine where abundance lies, Thy self thy foe, to thy sweet self too cruel: Thou that art now the world's fresh ornament, And only herald to the gaudy spring, Within thine own bud buriest thy content, And, tender churl, mak'st waste in niggarding:   Pity the world, or else this glutton be,   To eat the world's due, by the grave and thee.                       2  When forty winters shall besiege thy brow, And dig deep trenches in thy beauty's field, Thy youth's proud livery so gazed on now, Will be a tattered weed of small worth held: Then being asked, where all thy beauty lies, Where all the treasure of thy lusty days; To say, within thine own deep sunken eyes, Were an all-eating shame, and thriftless praise. How 

In [0]:
chars = sorted(set(text))
charToint = {c:i for i, c in enumerate(chars)}
intTochar = np.array(chars)

In [10]:
#Instantiate sequence data
maxlen = 20
step = 10

encoded = [charToint[c] for c in text]

sequence = []
next_char = []

for x in range(0, len(encoded) - maxlen, step):
  sequence.append(encoded[x : x + maxlen])
  next_char.append(encoded[x + maxlen])
print(f'Sequence: {len(sequence)}')

Sequence: 557497


In [11]:
sequence[0]

[34, 77, 74, 72, 1, 65, 60, 68, 77, 64, 78, 79, 1, 62, 77, 64, 60, 79, 80, 77]

In [0]:
#Instantiate X and y
X = np.zeros((len(sequence) , maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sequence), len(chars)), dtype = np.bool)

for i, s in enumerate(sequence):
  for t, char in enumerate(s):
    X[i,t,char] = 1

  y[i, next_char[i]] = 1

In [18]:
print(X.shape, y.shape)

(557497, 20, 106) (557497, 106)


In [0]:
#Build a single model LSTM
model = Sequential([
        LSTM(128, input_shape=(maxlen, len(chars))),
        Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
#Helper function
def sample(preds):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / 1
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
#Invoking function at the end of each epoch
def epoch_end(epoch, _):
  print()
  print('----- Generating text after Epoch: %d' % epoch)

  start_index = random.randint(0, len(text) - maxlen - 1)

  generated = ''
  sentence = text[start_index: start_index + maxlen]
  generated += sentence 

  print('-----Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)

  for i in range(50):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, charToint[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = intTochar[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()
print_callback = LambdaCallback(on_epoch_end=epoch_end)

In [34]:
#fit the model
model.fit(X,y,
          batch_size=32,
          epochs=2,
          callbacks=[print_callback])

Epoch 1/2
17422/17422 [==============================] - ETA: 0s - loss: 1.6648 - accuracy: 0.5160
----- Generating text after Epoch: 0
-----Generating with seed: " I will kiss thy foo"
 I will kiss thy foolt man's tree. Do sujed concrany     With year'st 
17422/17422 [==============================] - 387s 22ms/step - loss: 1.6648 - accuracy: 0.5160
Epoch 2/2
17421/17422 [============================>.] - ETA: 0s - loss: 1.6182 - accuracy: 0.5276
----- Generating text after Epoch: 1
-----Generating with seed: "se our utmost studie"
se our utmost studier to deit say! I have him when to your charmens am
17422/17422 [==============================] - 395s 23ms/step - loss: 1.6182 - accuracy: 0.5276


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN